# Web Scrapping Classwork

The purpose is to scrap all the pages returned with the entered keyword on pdfdrive  

Richard Luke, 2022/06/05

In [1]:
#importing libraries to use for scrapping
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#for ML
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
#function to start a search
search=input("Enter the Book Category to Search : ")
def load(search,number):
    try:
        # + str(page_num)
        URL = f"https://www.pdfdrive.com/search?q={search}&page={number}"
    except:
        print("SORRY! Category not found.")
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    return soup

def getdata():
    titles = []
    pages = []
    year = []
    size = []
    downloads = []
    
    try:
        for result in soup.find_all('div', attrs={'class': 'file-right'}):
            title = result.find("h2").text
            titles.append(title)

        for pag in soup.find_all('div', attrs={'class': 'file-info'}):
            page = pag.find("span", class_="fi-pagecount").text
            years = pag.find("span", class_="fi-year").text
            file_size = pag.find("span", class_="fi-size hidemobile").text
            down = pag.find("span", class_="fi-hit").text
            pages.append(page)
            year.append(years)
            size.append(file_size)
            downloads.append(down)

        my_scrapper = {"Book Name": titles,
                    "No of Pages": pages,
                    "Year": year,
                     "FILE SIZE (MB)": size,
                     "DOWNLOADS (Count)": downloads
                    }
        df = pd.DataFrame(my_scrapper)
        df.to_csv('/Users/Richard/Downloads/PDFdrive list.csv', mode='a', header=False,index=False)
        return df
    except:
        print("Failed to append")


def appenddata(df):
    try:
              # split 
        df[["Number.Pages", "Pages"]]=df["No of Pages"].str.split(expand=True)
        df[["Number.Fsize", "File Size"]]=df["FILE SIZE (MB)"].str.split(expand=True)
        df[["Number.Downloads", "Downloads"]]=df["DOWNLOADS (Count)"].str.split(expand=True)
        # prepare string data type for conversion to int
        df['Number.Downloads']=df['Number.Downloads'].str.replace(',','')
        df["Number.Pages"]=df["Number.Pages"].str.replace(',','')
        # drop
        del df["No of Pages"]
        del df["FILE SIZE (MB)"]
        del df["DOWNLOADS (Count)"]

        #save
        df.to_csv('/Users/Richard/Downloads/PDFdrive cleaned list.csv',mode='a', index=False)

    except:
        print('Search returned nothing!')
        
    return df

def cleandata():
    data=pd.read_csv('/Users/Richard/Downloads/PDFdrive cleaned list.csv')
    #remove strings
    try:
         data = data[data["Number.Fsize"].str.contains("Pages|Number.Downloads|Number.Fsize") == False]
    except:
        pass
    # change data type
    data["Number.Pages"]=data["Number.Pages"].astype(str).astype(int)
    data["Year"]=data["Year"].astype(str).astype(int)
    data['Number.Downloads']=data['Number.Downloads'].astype(str).astype(int)
    data["Number.Fsize"]=data["Number.Fsize"].astype(str).astype(float)
    #save to df
    return data

while True:
    number=1
    soup=load(search,str(number))
    sleep(15)
    df=getdata()
    sleep(15)
    df=appenddata(df)
    number=number+1
    #sleep(15)

Enter the Book Category to Search : data science


In [ ]:
data=cleandata()
data.head()

In [ ]:
data.info()

In [ ]:
round(data.describe(),1)

In [ ]:
plt.boxplot(data["Number.Downloads"], vert=False),
plt.title("Number of Downloads");

In [ ]:
plt.boxplot(data["Number.Pages"], vert=False),
plt.title("Number of Pages");

In [ ]:
low, high =data["Number.Downloads"].quantile([0.1,0.75])

mask_downloads=data["Number.Downloads"].between(low,high)
data=data[mask_downloads]
round(data.describe(),2)

In [ ]:
plt.hist(data["Number.Downloads"]);

#round(df["Number.Downloads"].quantile([0.1,0.75]).describe(),2)

In [ ]:
plt.scatter(data["Number.Pages"],data["Number.Downloads"]),
plt.title("Book Size vs Number of Downloads"),
plt.xlabel("Number of Pages"),
plt.ylabel("Number of Downloads");

In [ ]:
#Relationship between File Size and Number of Downloads
plt.scatter(data["Number.Fsize"],data["Number.Downloads"]),
plt.title("File Size vs Number of Downloads"),
plt.xlabel("File Size (MB)"),
plt.ylabel("Number of Downloads");

In [ ]:
#Relationship between Number of Downloads and Year of Publication
plt.bar(
    data["Year"],data["Number.Downloads"]),
plt.title("Year vs Number of Downloads"),
plt.xlabel("Year"),
plt.ylabel("Number of Downloads");


In [ ]:
print("The Most Downloaded Book is \n",data["Book Name"].max())

In [ ]:
print("The Least Downloaded Book is \n",data["Book Name"].min())

In [ ]:
#Logistic regression

data.info()